In [1]:

####################################################################################################
####################################################################################################

import sys
import os.path

import datetime
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd

from IPython.display import display
from IPython.display import Image

sys.path.append('/project/lgrandi/dbarge/')

print()
print("-> Python Executable (sys.executable):   " + sys.executable)
print("-> Python Path (sys.path):")
for x in sys.path: print("   " + x)
print()


####################################################################################################
####################################################################################################

import pax
print("-> Pax version: " + pax.__version__)


####################################################################################################
####################################################################################################


from pax_utils import s1s2_utils
from pax_utils.helpers import *


####################################################################################################
####################################################################################################

import tensorflow as tf

print ("-> Tensorflow version:   " + tf.__version__ + "\n")

import keras
import keras.utils.vis_utils
from keras import backend as K
from keras import layers
from keras import regularizers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.models import Sequential
from keras.utils import plot_model


####################################################################################################
####################################################################################################

from models import *


####################################################################################################
####################################################################################################

v_drift = 1.44e-4

print("Ready")



-> Python Executable (sys.executable):   /project/lgrandi/dbarge/anaconda3/envs/ml_py365/bin/python
-> Python Path (sys.path):
   
   /project/lgrandi/dbarge/pax_v6.5.1
   /home/dbarge/reconstruction/z_s2waveforms
   /project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python36.zip
   /project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python3.6
   /project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python3.6/lib-dynload
   /project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python3.6/site-packages
   /project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python3.6/site-packages/Mako-1.0.7-py3.6.egg
   /project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python3.6/site-packages/IPython/extensions
   /home/dbarge/.ipython
   /project/lgrandi/dbarge/

-> Pax version: 6.5.1
-> Tensorflow version:   1.10.0



Using TensorFlow backend.


Ready


In [6]:

####################################################################################################
####################################################################################################

nEventsTrain = 100000
file_all     = '/project/lgrandi/dbarge/simulation/wimp/pax_v6.8.3/merged/merged_all_200000.pkl'

df_all = pd.read_pickle(file_all)
df_all = df_all[df_all['intr_count'] == 1].reset_index(drop=True)
df_all = df_all[nEventsTrain:][:]

df_all['intr_depth'] = v_drift*df_all['intr_drift_time'] 


####################################################################################################
####################################################################################################

df_train_truth = df_all[:][['depth']]
df_train_input = df_all[:][['intr_depth']]


####################################################################################################
####################################################################################################

checkTrainingData(df_train_input, df_train_truth)




Input shape: (27826, 1)
Truth shape: (27826, 1)

Input DataFrame: 


,intr_depth
100000,89.66448
100001,77.97456
100002,63.97920
100003,79.48512
100004,87.67728



Truth DataFrame:


,depth
100000,89.428794
100001,77.495845
100002,63.686664
100003,79.201652
100004,87.226589


In [7]:

####################################################################################################
####################################################################################################

model_name     = "model_barge_z_elu"
folder         = "models" + "/"    
model_name_h5  = folder + model_name + ".h5"
model_name_png = folder + model_name + ".png"



In [8]:

####################################################################################################
####################################################################################################

model       = load_model(model_name_h5)
arr_xy_pred = model.predict(df_train_input)

print("\nLoaded Model: " + model_name_h5)
print("Predicted\n")




Loaded Model: models/model_barge_z_elu.h5
Predicted



In [9]:

####################################################################################################
####################################################################################################

arr_z_true = df_train_truth[:]['depth'].as_matrix()
arr_z_pred = arr_xy_pred[:, 0]
arr_dz     = arr_z_true - arr_z_pred



/project/lgrandi/dbarge/anaconda3/envs/ml_py365/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [14]:

####################################################################################################
####################################################################################################

file_out         = "./predictions/z_DeltaT.pkl"

cols_out         = ['event_number', 'depth', 'intr_depth', 'intr_z']

df_out           = df_all[:][cols_out]
df_out['z_pred'] = arr_z_pred


df_out.to_pickle(file_out)


####################################################################################################
####################################################################################################

df_test = pd.read_pickle(file_out)

display(df_test[0:5][:])
print()



,event_number,depth,intr_depth,intr_z,z_pred
100000,156467,89.428794,89.66448,-89.41968,89.281265
100001,156468,77.495845,77.97456,-77.72976,77.599518
100002,156469,63.686664,63.97920,-63.73440,63.613926
100003,156470,79.201652,79.48512,-79.24032,79.109024
100004,156474,87.226589,87.67728,-87.43248,87.295448
